<a href="https://colab.research.google.com/github/jooeun921/Big-Data-Analyst/blob/main/Part06_Section_01_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 06. 선형 모형
Python을 활용한 통계 모형을 다룸.
- 선형 회귀 분석 : 회귀모델 적합, 계수 해석, 다중공선성 문제(VIF), 모델평가(R^2, F-검정)
- 로지스틱 회귀 분석 : 오즈비(Odds Ratio), 계수 해석, 모델 유의성 검정(Deviance 검정)

### Section 01 학습 : 선형 회귀 분석

In [ ]:
# 피어슨 상관계수
import numpy as np
from scipy.stats import pearsonr

x = np.array([10, 20, 30, 40, 50])
y = np.array([5, 15, 25, 35, 48])

corr_coeff, p_value = pearsonr(x, y)
print(f"피어슨 상관계수 (r) = {corr_coeff}")
print(f"p_value = {p_value}")

In [ ]:
# 단순 선형 회귀

import numpy as np

x = np.array([10, 20, 30, 40, 50])
y = np.array([5, 15, 25, 35, 48])

x_mean = x.mean()
y_mean = y.mean()

# Sxy  = x와 y의 변동량. x가 변할 때, y도 변하는지.
Sxy = np.sum((x - x_mean) * (y - y_mean))
# Sxx = x의 변동량. x 분산 기반.
Sxx = np.sum((x - x_mean) ** 2)
# 기울기 = y의 변화량 / x의 변화량
beta_1 = Sxy / Sxx

# 절편 => y = ax + b
# b = y - ax
beta_0 = y_mean - beta_1 * x_mean

print(f"기울기 (beta 1) : {beta_1:.4f}")
print(f"절편 (beta 0) : {beta_0:.4f}")

print(f"회귀 직선 방정식 : y = {beta_0:.4f} + {beta_1:.4f} * x")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 예측값 계산
y_pred = beta_0 + beta_1 * x

# 시각화
plt.figure(figsize=(6, 4))
plt.scatter(x, y, label="real", color="blue")
plt.plot(x, y_pred, color="red", label="fitted line")
plt.title("fitted line visualization")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr

x = np.array([10, 20, 30, 40, 50])
y = np.array([5, 15, 25, 35, 48])

x_mean, y_mean = x.mean(), y.mean()
s_x, s_y = np.std(x, ddof = 1), np.std(y, ddof = 1)

r, _ = pearsonr(x, y)

beta_1 = r * (s_y / s_x)
beta_0 = y_mean - beta_1 * x_mean

print(f"상관계수 {r}")
print(f"기울기 beta 1 (r * sy/sx): {beta_1}")
print(beta_0)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

df_iris = load_iris()

iris = pd.DataFrame(data = df_iris.data, columns = df_iris.feature_names)
iris.columns = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width']

iris["species"] = df_iris.target
iris["species"] = iris["species"].map({0: "setosa", 1: "versicolor", 2: "virginica"})

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols("Petal_Length ~ Petal_Width + Sepal_Length", data = iris).fit()
print(model.summary())

In [ ]:
model = smf.glm("Petal_Length ~ Petal_Width + Sepal_Length", family = sm.families.Gaussian(), data = iris).fit()
print(model.summary())

In [ ]:
model = smf.ols("Petal_Length ~ Petal_Width + Sepal_Length + C(species)", data = iris).fit()
print(model.summary)
print(model.params)

In [ ]:
model = smf.ols("Petal_Length ~ Petal_Width + Sepal_Length", data = iris).fit()
# print(model.pvalues)
# print(model.tvalues)

#유의수준 5%
print(model.conf_int(alpha = 0.05))

# 유의수준 10%
print(model.conf_int(alpha = 0.10))

In [ ]:
print(model.summary())

In [ ]:
# 선형회귀모델 간 비교를 해야 할 때,

from statsmodels.formula.api import ols
import statsmodels.api as sm

model1 = ols("Petal_Length ~ Petal_Width", data = iris).fit()
model2 = ols("Petal_Length ~ Petal_Width + Sepal_Length + Sepal_Width", data = iris).fit()

table = sm.stats.anova_lm(model1, model2)
print(table)

In [ ]:
dw_stat = model.summary().tables[2].data[0][3]
print(model.summary())
print(dw_stat)

In [ ]:
residuals = model.resid
from scipy.stats import shapiro

sw_stat, sw_p_value = shapiro(residuals)

print(f"Statistic :  {sw_stat}")
print(f"p value : {sw_p_value}")

In [ ]:
# 잔차의 등분산 검정 방법.
from statsmodels.stats.diagnostic import het_breuschpagan

# import statsmodels.stats

bptest = het_breuschpagan(model.resid, model.model.exog)

print("BP-test statistic", bptest[0])
print("p-value", bptest[3])

# print(help(het_breuschpagan))

In [ ]:
# 다중공선성

from statsmodels.stats.outliers_influence import variance_inflation_factor

X = iris[['Petal_Width', 'Sepal_Length']]

vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

In [ ]:
# 예측

from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as sf

model = sf.ols('Petal_Length ~ Petal_Width + Sepal_Length + C(species)', data = iris).fit()

new_data = pd.DataFrame({
'Petal_Width': [0.2, 1.5, 1.3, 2.1, 1.8],
'Sepal_Length': [4.9, 5.5, 6.1, 6.7, 7.2],
'species': ['setosa', 'versicolor', 'virginica', 'versicolor', 'virginica']})

y_pred = model.predict(new_data)
y_true = np.array([1.4, 4.7, 5.1, 5.8, 6.3])

mse = mean_squared_error(y_true, y_pred)

print(y_pred)
print(mse)

### Section 01 연습문제 : 선형 회귀 분석

```
from statsmodels.formula.api import ols

model = ols('y~x1 + x2 + C(범주형)', data = data_set).fit()
model.summary()
model.params # 변수별 회귀계수
model.rsquared # 결정계수
model.pvalues # 유의확률
model.tvalues
model.resid # 잔차

model.predict(new_df) # 새로운 데이터 예측하기

# 데이터 프레임에서의 상관계수 구하기
df.corr()
```

In [ ]:
#1-3 다음은 다중 선형회귀 모델을 적합하기 위한 데이터이다.

import pandas as pd
import numpy as np

# 예제 데이터 생성
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 5)
y = 3 * X[:, 0] + 2 * X[:, 1] + X[:, 2] + np.random.randn(n_samples)
df = pd.DataFrame(X, columns = ['var1', 'var2', 'var3', 'var4', 'var5'])
df['target'] = y

# 데이터 확인
print(df.head())

In [ ]:
#1 target 변수와 가장 큰 상관 관계를 갖는 변수의 상관계수를 구하시오.

print(df.corr())
print("var1 : ", 0.687853)

In [ ]:
#2 다중 선형회귀 모양으로 target 변수를 예측할 때, 모델의 결정계수를 계산하시오.

from statsmodels.formula.api import ols

model = ols("target ~ var1 + var2 + var3 + var4 + var5", data = df).fit()
print(model.rsquared)

In [ ]:
model.summary()

In [ ]:
#3 앞에서 사용된 모델의 계수 검정에서 p-value가 가장 큰 변수와 그 값을 구하시오.

print(model.pvalues)
print("===================")
print(model.pvalues.idxmax())
print(model.pvalues.max())

In [ ]:
#4-6 다음은 다중 선형회귀 모델을 적합하기 위한 데이터이다.
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
import statsmodels.api as sm

# 예제 데이터 생성
X, y = make_regression(n_samples = 100, n_features = 3, noise = 0.1, random_state = 42)
df = pd.DataFrame(X, columns = [f'var{i}' for i in range(3)])
df['target'] = y

# 데이터 확인
print(df.head( ))

In [ ]:
#4 ❗ 유의확률(p-value)가 가장 작은 변수의 회귀계수를 구하시오. = model.params

from statsmodels.formula.api import ols
model2 = ols("target ~ var0 + var1 + var2", data = df).fit()

print(model2.pvalues.idxmin())

print(model2.params)
# print(75.0508)

In [ ]:
#5 적합한 회귀모델의 결정계수를 구하시오.

print(model2.rsquared)

In [ ]:
#6 적합한 회귀모델을 사용하여 var0 변수가 0.5, var1은 1.2, var2는 0.3일 때 예측값을 계산하시오.

import pandas as pd

new_data = pd.DataFrame({'var0' : [0.5], 'var1' : [1.2], 'var2' : [0.3]})

pred_y = model2.predict(new_data)
print(pred_y[0])